<a href="https://colab.research.google.com/github/jessedeveloperinvestor/AI-ML-Cheatsheets/blob/main/Cloud_Storage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ☁️ Cloud Computing com Google Colab + GCP

### 🔍 Coleta de Citações via API + Armazenamento no Cloud Storage + Ingestão no BigQuery

**Autor:** Fernando Nakamuta  
**Projeto:** poli-usp-pro-cloud-computing  
**Dataset:** `poliusp_demo.quotes`  
**Serviços utilizados:** `requests`, `Cloud Storage`, `BigQuery`

# Utilizando Cloud Storage


In [ ]:
# Imports
from google.colab import auth
from google.cloud import storage

#### Autenticação no Google Cloud

In [ ]:
auth.authenticate_user()

In [ ]:
project_id = "poli-usp-pro-cloud-computing"
client = storage.Client(project=project_id)

#### Fazendo download de arquivos de um bucket

In [ ]:
bucket_1 = client.get_bucket('poliusp-demo')
blob = bucket_1.blob('iris.csv')
blob.download_to_filename('iris_local.csv')

In [ ]:
bucket_2 = client.get_bucket('poliusp-pro-exemplo-aula')
blob = bucket_2.blob('iris.csv')
blob.upload_from_filename("iris_local.csv")

#### Fazendo upload de arquivos para um bucket

In [ ]:
with open("arquivo_teste.txt", "w") as f:
    f.write("Olá, nuvem! Este é um upload via Colab.")

In [ ]:
blob = bucket_2.blob("test/arquivo_teste.txt")  # nome do arquivo no GCS
blob.upload_from_filename("arquivo_teste.txt") # nome do arquivo local

print("Upload concluído com sucesso!")

Upload concluído com sucesso!


#### Fazendo download de arquivos de um bucket

In [ ]:
import requests
import json

In [ ]:
def extrai_quotes(pagina):
    url = f"https://quotes.toscrape.com/api/quotes?page={pagina}"
    headers = {
        "User-Agent": "Chrome/92.0.4515.159",
        "Accept": "application/json",
        "X-Requested-With": "XMLHttpRequest"
    }
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    return resp.json()

dados_pagina1 = extrai_quotes(1)
# print(json.dumps(dados_pagina1, indent=2, ensure_ascii=False))
dados_pagina1

{'has_next': True,
 'page': 1,
 'quotes': [{'author': {'goodreads_link': '/author/show/9810.Albert_Einstein',
    'name': 'Albert Einstein',
    'slug': 'Albert-Einstein'},
   'tags': ['change', 'deep-thoughts', 'thinking', 'world'],
   'text': '“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”'},
  {'author': {'goodreads_link': '/author/show/1077326.J_K_Rowling',
    'name': 'J.K. Rowling',
    'slug': 'J-K-Rowling'},
   'tags': ['abilities', 'choices'],
   'text': '“It is our choices, Harry, that show what we truly are, far more than our abilities.”'},
  {'author': {'goodreads_link': '/author/show/9810.Albert_Einstein',
    'name': 'Albert Einstein',
    'slug': 'Albert-Einstein'},
   'tags': ['inspirational', 'life', 'live', 'miracle', 'miracles'],
   'text': '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”'},
  {'author': {'goodreads_

In [ ]:
def coleta_todas_quotes():
    pagina = 1
    todas_quotes = []
    while True:
        data = extrai_quotes(pagina)
        todas_quotes.extend(data["quotes"])
        if not data["has_next"]:
            break
        pagina += 1
    return todas_quotes

# Coleta completa
citacoes = coleta_todas_quotes()
print(f"Total de citações coletadas: {len(citacoes)}")

Total de citações coletadas: 100


In [ ]:
# Função para extrair uma página
def extrai_quotes(pagina):
    url = f"https://quotes.toscrape.com/api/quotes?page={pagina}"
    headers = {
        "User-Agent": "Chrome/92.0.4515.159",
        "Accept": "application/json",
        "X-Requested-With": "XMLHttpRequest"
    }
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    return resp.json()

# Função principal: salva e envia cada página
def coleta_e_envia_por_pagina():
    pagina = 1
    total_quotes = 0

    while True:
        data = extrai_quotes(pagina)
        quotes = data["quotes"]
        total_quotes += len(quotes)

        # Salvar localmente
        nome_arquivo = f"quotes_pagina_{pagina}.json"
        with open(nome_arquivo, "w", encoding="utf-8") as f:
            for quote in quotes:
                f.write(json.dumps(quote, ensure_ascii=False) + "\n")


        # Enviar para GCS
        blob_path = f"quotes/{nome_arquivo}"
        blob = bucket_2.blob(blob_path)
        blob.upload_from_filename(nome_arquivo)
        print(f"Página {pagina} enviada com sucesso para: gs://poliusp-demo/{blob_path}")

        if not data["has_next"]:
            break
        pagina += 1

    print(f"\nTotal de páginas: {pagina}")
    print(f"Total de citações coletadas: {total_quotes}")

# Executa
coleta_e_envia_por_pagina()

Página 1 enviada com sucesso para: gs://poliusp-demo/quotes/quotes_pagina_1.json
Página 2 enviada com sucesso para: gs://poliusp-demo/quotes/quotes_pagina_2.json
Página 3 enviada com sucesso para: gs://poliusp-demo/quotes/quotes_pagina_3.json
Página 4 enviada com sucesso para: gs://poliusp-demo/quotes/quotes_pagina_4.json
Página 5 enviada com sucesso para: gs://poliusp-demo/quotes/quotes_pagina_5.json
Página 6 enviada com sucesso para: gs://poliusp-demo/quotes/quotes_pagina_6.json
Página 7 enviada com sucesso para: gs://poliusp-demo/quotes/quotes_pagina_7.json
Página 8 enviada com sucesso para: gs://poliusp-demo/quotes/quotes_pagina_8.json
Página 9 enviada com sucesso para: gs://poliusp-demo/quotes/quotes_pagina_9.json
Página 10 enviada com sucesso para: gs://poliusp-demo/quotes/quotes_pagina_10.json

Total de páginas: 10
Total de citações coletadas: 100
